# Deducción de las ecuaciones diferenciales parciales de Cauchy-Navier en coordenadas polares para tensión plana

A continuación se presentarán dos verificaciones independientes de las ecuaciones de Cauchy-Navier en coordenadas polares para tensión plana:

In [1]:
from sympy import sin, cos, diff, Function, simplify, symbols, factor, Eq, init_printing, trigsimp, expand, latex, Matrix

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

In [2]:
r, t, z, x, y, E, nu = symbols("r, theta, z, x, y, E, nu")

# Se definen las constantes de Lamé
lamda = nu*E/((1 + nu)*(1 - 2*nu))
G     = E/(2*(1 + nu))

# Se definen los desplazamientos en coordenadas cilíndricas
ur = Function("u_r")(r, t)
vt = Function("v_θ")(r, t)
w  = Function("w")(r, t, z)

# Se definen las fuerzas másicas en coordenadas cilíndricas
br = Function("b_r")(r, t)
bt = Function("b_θ")(r, t)
bz = 0

In [3]:
# Matriz de transformación de la base {r,t,z} a la base {i,j,k}
T = Matrix([[ cos(t), -sin(t),       0 ],     
            [ sin(t),  cos(t),       0 ],      
            [      0,       0,       1 ]])

Se definen los desplazamientos en coordenadas rectangulares y cilíndricas:

In [4]:
# vector de desplazamientos con respecto a la base {r,t,z}
u_cil = Matrix([ ur,
                 vt,
                 w   ])

# vector de desplazamientos con respecto a la base {i,j,k}
u_rec = T * u_cil

# componentes de los desplazamientos en coordenadas rectangulares
u, v = u_rec[0], u_rec[1]

Se definen las fuerzas másicas en coordenadas rectangulares y cilíndricas:

In [5]:
# vector de fuerzas másicas con respecto a la base {r,t,z}
b_cil = Matrix([ br,     
                 bt,      
                 bz  ])

# vector de fuerzas másicas con respecto a la base {i,j,k}
b_rec = T * b_cil

# componentes de la fuerza másica en coordenadas rectangulares
X, Y, Z = b_rec

In [6]:
# se definen las primeras derivadas
d_dx   = lambda f: diff(f,r)*cos(t) - diff(f,t)*sin(t)/r
d_dy   = lambda f: diff(f,r)*sin(t) + diff(f,t)*cos(t)/r
d_dz   = lambda f: diff(f,z)

# se definen las segundas derivadas
d2_dx2 = lambda f: d_dx(d_dx(f))
d2_dy2 = lambda f: d_dy(d_dy(f))

Se define la divergencia de un campo vectorial en coordenadas rectangulares y polares:

In [7]:
# divergencia de un campo vectorial en coordenadas rectangulares y cilíndricas
div_rec = lambda v: d_dx(v[0]) + d_dy(v[1])                              # + d_dz(v[2])

div_pol = lambda v: diff(v[0],r) + v[0]/r + (1/r)*diff(v[1],t)           # + diff(v[2],z)

# Observe que en la definición de la divergencia para el caso bidimensional no se debe
# tener en cuenta la derivada diff(v[2],z). Si se hace, en el método 1 no se calculan 
# correctamente las ecuaciones diferenciales de Cauchy-Navier

Se define el laplaciano de una función en coordenadas rectangulares y cilíndricas:

In [8]:
# laplaciano en coordenadas rectangulares y cilíndricas para el caso bidimensional
lapl_rec = lambda f : simplify(d2_dx2(f) + d2_dy2(f))

lapl_pol = lambda f : simplify((1/r)*diff(r*diff(f,r),r) + (1/(r**2))*diff(f,t,2))

Se calcula la dilatación cúbica usando las fórmulas en coordenadas rectangulares y cilíndricas:

In [9]:
# se calcula la dilatación cúbica
e_rec = expand(div_rec(u_rec).subs({sin(t): 0, cos(t): 1}))
e_pol = expand(div_pol(u_cil).subs({sin(t): 0, cos(t): 1}))

# y se verifica que ambas expresiones sean iguales
e_rec - e_pol

## Método 1: convirtiendo las ecuaciones de Cauchy-Navier de coordenadas rectangulares

Se escriben las ecuaciones de Cauchy-Navier en coordenadas rectangulares:
$$
\begin{align}
 G \nabla^2 u + \frac{E}{2(1-\nu)} \frac{\partial}{\partial x}\left(\frac{\partial u }{\partial x} + \frac{\partial v}{\partial y}\right) + X &= 0 \\
 G \nabla^2 v + \frac{E}{2(1-\nu)} \frac{\partial}{\partial y}\left(\frac{\partial u }{\partial x} + \frac{\partial v}{\partial y}\right) + Y &= 0. 
\end{align} 
$$

In [10]:
eqCN_rec = Matrix([ G*(lapl_rec(u)) + (E/(2*(1 - nu)))*d_dx(d_dx(u) + d_dy(v)) + X,
                    G*(lapl_rec(v)) + (E/(2*(1 - nu)))*d_dy(d_dx(u) + d_dy(v)) + Y  ])

eqCN_rec = Matrix(eqCN_rec).subs({sin(t): 0, cos(t): 1})

Se escriben las ecuaciones de Cauchy-Navier en coordenadas polares:
$$
\begin{align}
    \frac{E}{2(1-\nu)} \frac{\partial e}{\partial r} + G\left(\nabla^2 u_r - \frac{u_r}{r^2} - \frac{2}{r^2} \frac{\partial v_\theta}{\partial \theta} \right) + b_r &= 0 \\
    \frac{E}{2(1-\nu)} \frac{1}{r} \frac{\partial e}{\partial \theta} + G\left(\nabla^2 v_\theta - \frac{v_\theta}{r^2} + \frac{2}{r^2} \frac{\partial u_r}{\partial \theta} \right) + b_\theta&= 0.
\end{align}
$$

In [11]:
eqCN_pol = Matrix([ (E/(2*(1 - nu)))*diff(e_pol,r)   + G*(lapl_pol(ur) - (ur/(r*r)) - (2/(r*r))*diff(vt, t)) + br,
                    (E/(2*(1 - nu)))*diff(e_pol,t)/r + G*(lapl_pol(vt) - (vt/(r*r)) + (2/(r*r))*diff(ur, t)) + bt  ])

Se verifica que ambos conjuntos de ecuaciones sean equivalentes:

In [12]:
simplify(eqCN_rec - eqCN_pol)

## Método 2: reemplazando la ley de Hooke en términos de desplazamientos las ecuaciones diferenciales de equilibrio

Se definen las deformaciones para coordenadas polares:
$$
\begin{align}
        \varepsilon_r(r,\theta) &= \frac{\partial u_r(r,\theta)}{\partial r} \\
        \varepsilon_\theta(r,\theta) &= \frac{u_r(r,\theta)}{r}  + \frac{1}{r}\frac{\partial v_\theta(r,\theta)}{\partial \theta} \\
        \varepsilon_z(r,\theta,z) &= \begin{cases}
            \frac{\partial w(r, \theta, z)}{\partial z} & \text{en tensión plana}     \\
                                                      0 & \text{en deformación plana}                                  
        \end{cases} \\
        \gamma_{r\theta}(r,\theta)  &= \frac{1}{r}\frac{\partial u_r(r,\theta)}{\partial \theta} + \frac{\partial v_\theta(r,\theta)}{\partial r} - \frac{v_\theta(r,\theta)}{r} \\
         \gamma_{rz}(r,\theta) &= 0 \\
        \gamma_{\theta z}(r,\theta) &= 0.
\end{align}
$$

In [13]:
er  = diff(ur,r)
et  = ur/r + diff(vt,t)/r
grt = diff(ur,t)/r + diff(vt,r) - vt/r

Se define la ley de Hooke para coordenadas polares en tensión plana:
$$
\begin{align}
\sigma_r      &= \frac{E}{1-\nu^2} \left(\varepsilon_r + \nu \varepsilon_\theta \right) & \tau_{r\theta}  &= G \gamma_{r\theta} \\
\sigma_\theta &= \frac{E}{1-\nu^2} \left(\varepsilon_\theta + \nu \varepsilon_r \right) & \tau_{rz}       &= 0 \\
\sigma_z      &= 0                                                                      & \tau_{\theta z} &= 0.
\end{align}
$$

In [14]:
sr  = (E/(1 - nu*nu))*(er + nu*et)
st  = (E/(1 - nu*nu))*(et + nu*er)
trt = G*grt

Se reemplazan las deformaciones en términos de desplazamiento en las ecuaciones de equilibrio para coordenadas polares:
$$
\begin{align}
 \frac{\partial \sigma_r(r,\theta)}{\partial r} + \frac{1}{r}\frac{\partial \tau_{r\theta}(r,\theta)}{\partial \theta} + \frac{\sigma_r(r,\theta) - \sigma_\theta(r,\theta)}{r} + b_r(r,\theta) &= 0 \\
 \frac{\partial \tau_{r\theta}(r,\theta)}{\partial r} + \frac{1}{r}\frac{\partial \sigma_\theta(r,\theta)}{\partial \theta}  + \frac{2 \tau_{r\theta}(r,\theta)}{r} + b_\theta(r,\theta) &= 0
\end{align}
$$

In [15]:
eqEQ_pol = Matrix([ diff(sr,r)  + diff(trt,t)/r + (sr - st)/r + br,
                    diff(trt,r) + diff(st,t)/r  + 2*trt/r     + bt ])

y de este modo se obtienen las ecuaciones diferenciales de Cauchy-Navier para el caso de coordenadas polares:

In [16]:
simplify(eqCN_rec - eqEQ_pol)